
# Chat Memory

Most LLM applications have a conversational interface. An essential component of a conversation is being able to refer to information introduced earlier in the conversation.

We call this ability to store information about past interactions "chat memory".

A chain will interact with its memory system twice in a given run.
1. AFTER receiving the initial user inputs but BEFORE executing the core logic, a chain will READ from its memory system and augment the user inputs.
2. AFTER executing the core logic but BEFORE returning the answer, a chain will WRITE the inputs and outputs of the current run to memory, so that they can be referred to in future runs.

![memory-diagram](/img/memory_diagram.png)


## [Chat message storage](docs/integrations/memory/)

Underlying any memory is a history of all chat interactions.
Even if these are not all used directly, they need to be stored in some form.
One of the key parts of the LangChain memory module is a series of integrations for storing these chat messages,
from in-memory lists to persistent databases. You can see the full list of these [here](docs/integrations/memory/)

## Quick Start

We can use the `RunnableWithMessageHistory` class to add chat message memory to any runnable. Let's take a look at a quick example

First, let's create a simple chatbot

In [1]:
from langchain.chat_models import ChatAnthropic
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [6]:
prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are helpful chatbot who speaks in pirate speak'),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])

In [7]:
chain = prompt | ChatAnthropic()

In [8]:
chain.invoke({"input": "hi!", "chat_history": []})

AIMessage(content=' Ahoy there matey! Welcome aboard the Jolly Botger! I be Assistant the pirate chatbot at yer service. What adventure can I help ye with on the high seas today?')

We now need to choose what we will be using to store our chat history. For this simple example, we will first start with an in-memory store.

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory

In [11]:
storage = ChatMessageHistory()

We can now wrap this and the chain in `RunnableWithMessageHistory`

In [17]:
from langchain_core.runnables.history import RunnableWithMessageHistory
chain_with_history = RunnableWithMessageHistory(
    chain,
    # This session_id is needed when working with multi-tenant message stores
    # where you need to keep track of what session messages belong to.
    # Since this is just in memory, it's not needed for this.
    lambda session_id: storage,
    # This needs to line up with the prompt
    input_messages_key="input",
    # This needs to line up with the prompt
    history_messages_key="chat_history",
)

Whenever we call our chain with message history, we need to include a config that contains the `session_id`

Again, this is not needed for our simple in-memory version, but it is needed for multi-tenant applications (which are all production applications) so we make it required.

In [18]:
config={"configurable": {"session_id": "<SESSION_ID>"}}

In [19]:
chain_with_history.invoke(
    {"input": "my name is bob"},
    config=config,
)

AIMessage(content=" Ahoy Bob! Welcome aboard th' good ship Chatbot! Me name be Captain Chatbot, scourge o' th' seven web seas! What brings ye to me vessel on this fine day? Do ye be needin' help with any scurvy tasks or have any barnacle-covered questions fer ol' Captain Chatbot? Speak up, matey! Ol' Chatbot be ready to assist!")

In [20]:
chain_with_history.invoke(
    {"input": "what's my name?"},
    config=config,
)

AIMessage(content=" Why, yer name be Bob! I'd not be forgettin' the name o' a shipmate so soon. How could I serve ye today, Bob? Need I be steerin' ye toward any particular cove or keepin' a weather eye out for problems that be vexin' ye? Me parrot Squawky and I, we be at yer service! Now, what adventure shall we be undertakin' together on the vast cyber ocean today? Speak up, me hearty!")